In [4]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import torchvision
torchvision.disable_beta_transforms_warning()

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# ollama
Settings.llm = Ollama(model="llama3.3", request_timeout=360.0, temperature=0)

In [5]:
from llama_index.core.tools import FunctionTool
from llama_index.core.tools import QueryEngineTool

In [6]:
import os

pdf_dir = 'data'

query_engine_tools = []

# Iterate through each PDF in the directory
for file_name in os.listdir(pdf_dir):
    if file_name.endswith(".pdf"):
        # Load the document
        file_path = os.path.join(pdf_dir, file_name)
        reader = SimpleDirectoryReader(input_files=[file_path])
        documents = reader.load_data()
        
        # Create an index for the document
        vector_index = VectorStoreIndex.from_documents(documents)
        query_engine_tool = QueryEngineTool.from_defaults(
            vector_index.as_query_engine(
                similarity_top_k=3,
            ),
            name=f"{file_name} query engine",
            description=f"Query engine for the American College of Radiology (ACR) appropriateness criteria on {file_name[:-4]}."
        )
        
        query_engine_tools.append(query_engine_tool)

In [7]:
fallback_tool = FunctionTool.from_defaults(
    lambda x: 'No relevant information contained in the American College of Radiology (ACR) appropriateness criteria',
    name="Fallback Tool",
    description="Handles queries no other tool can."
)

In [8]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgent, AgentRunner, FunctionCallingAgentWorker

In [11]:
agent = FunctionCallingAgent.from_tools(
    query_engine_tools + [fallback_tool],
    llm=Settings.llm,
    system_prompt='You are a specialized agent designed to answer queries about the planning and follow up \
    for topics related to interventional radiology based on the American College of Radiology (ACR) appropriateness criteria. \
    Answer using only information from one or more of the provided tools. \
    Do not answer from prior knowledge under any circumstances. \
    If no appropriate tool exists or if the tools do not provide sufficient context, do not answer the query. Instead, \
    indicate that the necessary information is not available in the ACR appropriateness criteria.',
    verbose=True
)

In [23]:
agent = FunctionCallingAgent.from_tools(
    query_engine_tools + [fallback_tool],
    llm=Settings.llm,
    system_prompt='You are a specialized agent designed to answer queries about the planning and follow up \
    for topics related to interventional radiology based on the American College of Radiology (ACR) appropriateness criteria.\
    Given clinical vignettes, you are able to tell me the most appropriate next step and why. \
    If you tell me the imaging, please be as specific as possible, including utilization of contrast or location of the imaging.\
    The PDFs you are searching have a clinical vignette, followed by a table telling you the level of \
    appropriateness of the intervention/imaging. What you need to pay attention to are the ones that say Usually Appropriate. \
    If you tell me multiple potential options or next steps, tell me which are more appropriate. If one or more options is "Usually Appropriate",\
    explicitly state so. Answer using only information from one or more of the provided tools.\
    Do not answer from prior knowledge under any circumstances. If no appropriate tool exists or if the tools do not provide sufficient\
    context, do not answer the query. Instead, indicate that the necessary information is not available in the ACR\
    appropriateness criteria. Cite your sources. Once you have an answer, stop',
    verbose=True
)

In [ ]:
response = agent.chat(
    'What kind of catheter should I use for central line? '
    'On a separate note, I also want to know if my patient with a DVT needs a stent. '
    'Also, do you know why blood is red? '
)

> Running step f54e48ef-5991-4078-ba22-b9fd157621e8. Step input: What kind of catheter should I use for central line? On a separate note, I also want to know if my patient with a DVT needs a stent. Also, do you know why blood is red? 
Added user message to memory: What kind of catheter should I use for central line? On a separate note, I also want to know if my patient with a DVT needs a stent. Also, do you know why blood is red? 
=== Calling Function ===
Calling function: Central Venous Access Device and Site Selection.pdf query engine with args: {"input": "What kind of catheter should I use for central line?"}
=== Function Output ===
The choice of catheter for central line placement depends on various factors such as the duration of therapy, patient condition, and intended use. For patients requiring renal replacement therapy for less than 2 weeks, a nontunneled dialysis catheter may be considered. However, for longer durations, a tunneled dialysis catheter is preferred due to lower 

In [22]:
response = agent.chat(
    '30 year old woman with fibroids who still desires pregnancy but is having trouble with fertility. What is the best next treatment plan?'
)

> Running step 866b1e54-b48f-4706-91e6-2ef9f043967f. Step input: 30 year old woman with fibroids who still desires pregnancy but is having trouble with fertility. What is the best next treatment plan?
Added user message to memory: 30 year old woman with fibroids who still desires pregnancy but is having trouble with fertility. What is the best next treatment plan?
=== Calling Function ===
Calling function: Management of Uterine Fibroids.pdf query engine with args: {"input": "30 year old woman with fibroids having trouble with fertility"}


KeyboardInterrupt: 

In [17]:
response = agent.chat(
    'Postmenopausal patient with uterine fibroids, symptomatic with heavy uterine bleeding or \
    bulk symptoms (eg, pressure, pain, fullness, bladder, or bowel symptoms). Negative \
    endometrial biopsy. Next step.'
)

> Running step f3d95d1d-be61-4c99-be41-bb110cb6d5a6. Step input: Postmenopausal patient with uterine fibroids, symptomatic with heavy uterine bleeding or     bulk symptoms (eg, pressure, pain, fullness, bladder, or bowel symptoms). Negative     endometrial biopsy. Next step.
Added user message to memory: Postmenopausal patient with uterine fibroids, symptomatic with heavy uterine bleeding or     bulk symptoms (eg, pressure, pain, fullness, bladder, or bowel symptoms). Negative     endometrial biopsy. Next step.
=== Calling Function ===
Calling function: Management of Uterine Fibroids.pdf query engine with args: {"input": "Postmenopausal patient with uterine fibroids, symptomatic with heavy uterine bleeding or bulk symptoms (eg, pressure, pain, fullness, bladder, or bowel symptoms). Negative endometrial biopsy."}
=== Function Output ===
For a postmenopausal patient with uterine fibroids who is symptomatic with heavy uterine bleeding or bulk symptoms and has a negative endometrial biopsy

In [ ]:
from llama_index.core import PromptTemplate

In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools + [fallback_tool],
    llm=Settings.llm,
    verbose=True
)

In [ ]:
react_system_header_str = """\

You are designed to help with a variety of tasks, from answering questions \
    to providing summaries to other types of analyses.

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:
{tool_desc}

## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information
to answer the question without using any more tools. At that point, you MUST respond
in the one of the following two formats:

```
Thought: I can answer without using any more tools.
Answer: [your answer here]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query.
```

## Additional Rules
- The answer MUST contain a sequence of bullet points that explain how you arrived at the answer. This can include aspects of the previous conversation history.
- You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.
- You are a specialized agent designed to answer queries about the planning and follow up for topics related to interventional radiology based on the American College of Radiology (ACR) appropriateness criteria.
- Answer using only information from one or more of the provided tools.
- Do not answer from prior knowledge under any circumstances.
- If no appropriate tool exists or if the tools do not provide sufficient context, do not answer the query.
- Instead, indicate that the necessary information is not available in the ACR appropriateness criteria.

## Current Conversation
Below is the current conversation consisting of interleaving human and assistant messages.

"""
react_system_prompt = PromptTemplate(react_system_header_str)

In [ ]:
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})

In [ ]:
response = agent.chat(
    'What kind of catheter should I use for central line? '
    'On a separate note, I also want to know if my patient with a DVT needs a stent. '
    'Also, do you know why blood is red? '
)

### Thoughts
- Can use as a training tool.
- Can be non-IR facing
- Can be IR facing

### Division of labor from here on out:
- one person starts user interface stuff
- one person makes the actual thing better (look into cohere api. josh said it was free)

### Other Thoughts
- I think this may be kinda shitty if we only include ACR AC. Should we think about incorporating other sources? If scaling up significantly, may need a way to filter tool selection. I think I saw how to do this somewhere.
- What is our big-picture goal with this? Could be good for pitch. e.g., who's target market
- Are there any benchmarks for this we can try? Could be nice to do a ChatGPT vs. our thing or something

### Other to dos
- query user for more info
- maintain history
- take in vignettes and also answer questions
- Cite sources/document used
- Gradio
- Decide on overall agentic rag pipeline/what works best in terms of which agents/how many

big picture todos:
1. performance
2. interface